In [6]:
from DbConnector import DbConnector
from part2 import Database 

In [7]:
# Set up the program
program = DbConnector()
db = Database()

### Task 1

How many users, activities and trackpoints are there in the dataset (after it is inserted into the database)

In [ ]:
# Get the number of Users
users = db.get_table_size("User")
print(f"There are {users[0]} users")

# Get the number of Activities
activities = db.get_table_size("Activity")
print(f"There are {activities[0]} activities")

# Get the number of TrackPoints
trackpoints = db.get_table_size("TrackPoint")
print(f"There are {trackpoints[0]} trackpoints")


There are (182,) users
There are (21057,) activities
There are (5355109,) trackpoints


### Task 3

Find the top 15 users with the highest number of activities.

In [ ]:
query = "SELECT user_id FROM Activity GROUP BY user_id ORDER BY COUNT(*) DESC LIMIT 15"
program.cursor.execute(query)
rows = program.cursor.fetchall()
text = "Top 15 users with the highest number of activities:\n"

for i, val in enumerate(rows):
    text += f"Top {i+1}. {val[0]}\n"
print(text)

Top 15 users with the highest number of activities:
Top 1. 163
Top 2. 085
Top 3. 153
Top 4. 068
Top 5. 167
Top 6. 128
Top 7. 062
Top 8. 025
Top 9. 075
Top 10. 126
Top 11. 010
Top 12. 052
Top 13. 084
Top 14. 041
Top 15. 004



### Task 4

Find all users who have taken a bus.

In [ ]:
# Get the users who have taken the bus
users = db.get_user_taken_bus()
users = "".join([f'\n user: {user[0]}' for user in users[0]])
print(f"The user ids for user that have taken the bus: {users}")

The user ids for user that have taken the bus: 
 user: 010
 user: 020
 user: 052
 user: 053
 user: 058
 user: 062
 user: 064
 user: 065
 user: 067
 user: 068
 user: 069
 user: 073
 user: 075
 user: 078
 user: 080
 user: 081
 user: 082
 user: 084
 user: 085
 user: 088
 user: 091
 user: 092
 user: 096
 user: 098
 user: 100
 user: 101
 user: 102
 user: 104
 user: 105
 user: 108
 user: 110
 user: 111
 user: 112
 user: 114
 user: 124
 user: 125
 user: 126
 user: 128
 user: 129
 user: 138
 user: 139
 user: 141
 user: 147
 user: 153
 user: 154
 user: 161
 user: 163
 user: 167
 user: 174
 user: 175
 user: 179


### Task 6

Find activities that are registered multiple times. You should find the query even
if it gives zero result.

In [ ]:
query = "SELECT a.id FROM Activity AS a WHERE EXISTS (SELECT b.id FROM Activity AS b WHERE a.user_id = b.user_id AND a.transportation_mode = b.transportation_mode AND a.start_date_time = b.start_date_time AND a.end_date_time = b.end_date_time AND a.id != b.id)"
program.cursor.execute(query)
rows = program.cursor.fetchall()
print("Find activities that are registered multiple times:\n", rows)

Find activities that are registered multiple times:
 []


### Task 7

a) Find the number of users that have started an activity in one day and ended the activity the next day.

In [ ]:
users = db.get_user_activity_over_a_day()

num = set()
for user in users[0]:
    num.add(user[0])

count = len(num)

print(f"Number of users with activity that ends the next day: {count}")

Number of users with activity that ends the next day: 95


b) List the transportation mode, user id and duration for these activities.

In [ ]:
users = "".join([f'\n user: {user[0]} transportation mode: {user[1]} duration: {user[2]}' for user in users[0]])
print(f"The user who have an end date one day after the start day: {users}")
print(users[0])

The user who have an end date one day after the start day: 
 user: 000 transportation mode: None duration: 4:09:15
 user: 000 transportation mode: None duration: 0:10:45
 user: 000 transportation mode: None duration: 11:03:55
 user: 000 transportation mode: None duration: 1:43:20
 user: 000 transportation mode: None duration: 16:24:06
 user: 001 transportation mode: None duration: 6:54:46
 user: 001 transportation mode: None duration: 0:09:30
 user: 001 transportation mode: None duration: 0:24:57
 user: 001 transportation mode: None duration: 14:17:28
 user: 001 transportation mode: None duration: 8:26:34
 user: 001 transportation mode: None duration: 1:19:41
 user: 001 transportation mode: None duration: 0:55:48
 user: 001 transportation mode: None duration: 0:58:08
 user: 001 transportation mode: None duration: 0:42:03
 user: 001 transportation mode: None duration: 13:30:14
 user: 001 transportation mode: None duration: 2:40:20
 user: 001 transportation mode: None duration: 0:46:26
 

### Task 9

Find the top 15 users who have gained the most altitude meters.

In [ ]:
query = "SELECT user_id, sum(activity_altitude)*0.304 as altitude_in_meters FROM Activity JOIN (SELECT activity_id, SUM(difference) AS activity_altitude FROM (SELECT activity_id, altitude - LAG(altitude) OVER (PARTITION BY activity_id ORDER BY date_time) AS difference FROM TrackPoint) AS altitude_difference WHERE difference > 0 GROUP BY activity_id) AS difference_table ON Activity.id = difference_table.activity_id GROUP BY user_id ORDER BY altitude_in_meters DESC LIMIT 15"
program.cursor.execute(query)
rows = program.cursor.fetchall()
text = "Find the top 15 users who have gained the most altitude meters\n"
for i in rows:
    text += f"User: {i[0]}, Altitude in meters: {i[1]}\n"
print(text)

Find the top 15 users who have gained the most altitude meters
User: 004, Altitude in meters: 331164.832User: 041, Altitude in meters: 240126.560User: 003, Altitude in meters: 233050.352User: 030, Altitude in meters: 175218.608User: 128, Altitude in meters: 152570.912User: 039, Altitude in meters: 146318.544User: 000, Altitude in meters: 121185.952User: 002, Altitude in meters: 114895.888User: 025, Altitude in meters: 108861.792User: 037, Altitude in meters: 98960.512User: 140, Altitude in meters: 94589.904User: 062, Altitude in meters: 74559.040User: 017, Altitude in meters: 62402.080User: 034, Altitude in meters: 61278.496User: 042, Altitude in meters: 61159.024


### Task 10

Find the users that have traveled the longest total distance in one day for each
transportation mode.

### Task 12

Find all the users who have registered transportation_mode and their most used transportation_mode

Comment on implementation: Although with this dataset, does not need to join User, however in cases where user does not have labels, but still has some activities in Activity with transportation_mode then a join is necessary. For users that have the same number of activities tagged on multiple transportation mode, we've decided to take the first transportation mode in alphabetical order.

In [ ]:
query = "SELECT user_id, transportation_mode FROM ( SELECT user_id, transportation_mode, ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY COUNT(*) DESC, transportation_mode ASC) AS rownum FROM Activity JOIN User ON Activity.user_id = User.id WHERE transportation_mode IS NOT NULL AND User.has_labels=TRUE GROUP BY user_id, transportation_mode) AS activity_grouped WHERE rownum=1 ORDER BY user_id"
program.cursor.execute(query)
rows = program.cursor.fetchall()
text = "Find all the users who have registered transportation_mode and their most used transportation_mode\n"
for row in rows:
    text += f"User {row[0]}, Transportation mode: {row[1]}\n"
print(text)

Find all the users who have registered transportation_mode and their most used transportation_mode
User 010, Transportation mode: walk
User 020, Transportation mode: bike
User 021, Transportation mode: car
User 052, Transportation mode: bus
User 053, Transportation mode: walk
User 056, Transportation mode: bike
User 058, Transportation mode: walk
User 059, Transportation mode: walk
User 060, Transportation mode: walk
User 062, Transportation mode: bus
User 064, Transportation mode: walk
User 065, Transportation mode: bike
User 067, Transportation mode: walk
User 068, Transportation mode: bike
User 069, Transportation mode: walk
User 073, Transportation mode: walk
User 075, Transportation mode: walk
User 076, Transportation mode: car
User 078, Transportation mode: walk
User 080, Transportation mode: walk
User 081, Transportation mode: walk
User 082, Transportation mode: walk
User 084, Transportation mode: walk
User 085, Transportation mode: walk
User 086, Transportation mode: car
User 0

### End

In [ ]:
# Closing the connection
program.close_connection()


-----------------------------------------------
Connection to 8.0.34-0ubuntu0.22.04.1 is closed
